In [1]:
import sys
sys.path.append("/home/chenlidong/polymer-chemprop-master/chemprop/")
from args import TrainArgs
from features import mol2graph, BatchMolGraph, get_atom_fdim, get_bond_fdim
from nn_utils import get_activation_function, index_select_ND
from models import MoleculeModel
from data import MoleculeDatapoint,MoleculeDataset
import pdb
import torch
from chemprop.utils import load_checkpoint
import os

import numpy as np
import torch
from rdkit import Chem
from rdkit.Chem.Draw import SimilarityMaps
import matplotlib.pyplot as plt

/home/chenlidong/.conda/envs/py_38_torch_113_pyg/lib/python3.8/site-packages/torch_geometric/typing.py:90: UserWarning: An issue occurred while importing 'torch-spline-conv'. Disabling its usage. Stacktrace: /lib64/libm.so.6: version `GLIBC_2.27' not found (required by /home/chenlidong/.conda/envs/py_38_torch_113_pyg/lib/python3.8/site-packages/torch_spline_conv/_basis_cuda.so)
  warnings.warn(


In [4]:
model = load_checkpoint("/home/chenlidong/polymer-chemprop-master/data/polymer-chemprop-data-main/results/vipea/cv9-monomer/master/checkpoints/chemprop_checkpoints_wDMPNN/0/12138/fold_0/model_0/model.pt",device=torch.device("cuda"))

Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_i.bias".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.bias".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Moving model to cuda


In [8]:
point = MoleculeDatapoint(smiles=["N c 1 n c ( C 2 C C ( N ) C 2 ) c c ( N 2 C C N ( C C ( = O ) N 3 C C C C 3 ) C C 2 ) n 1 . C C ( = O ) O C ( = O ) C S C C S C C ( = O ) O C ( C ) = O".replace(" ","")],
                 targets= [1.,2.],
                 ctype=0,
                 mtype=0,
                 data_weight= 1,
                #  features: np.ndarray = None,
                #  features_generator: List[str] = None,
                #  phase_features: List[float] = None,
                #  atom_features: np.ndarray = None,
                #  atom_descriptors: np.ndarray = None,
                #  bond_features: np.ndarray = None,
                #  overwrite_default_atom_features: bool = False,
                #  overwrite_default_bond_features: bool = False
                 )

In [9]:
batch = MoleculeDataset([point])

In [10]:
mol_batch, features_batch, target_batch, atom_descriptors_batch, atom_features_batch, bond_features_batch, data_weights_batch = \
            batch.batch_graph(), batch.features(), batch.targets(), batch.atom_descriptors(), \
            batch.atom_features(), batch.bond_features(), batch.data_weights()

In [11]:
model(mol_batch, features_batch, atom_descriptors_batch, atom_features_batch, bond_features_batch)

tensor([[3.6237, 0.1547]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [66]:


def visualize_atom_attention(viz_dir: str,
                             smiles: str,
                             num_atoms: int,
                             attention_weights: torch.FloatTensor,
                             prefix):
    """
    Saves figures of attention maps between atoms. Note: works on a single molecule, not in batch
    :param viz_dir: Directory in which to save attention map figures.
    :param smiles: Smiles string for molecule.
    :param num_atoms: The number of atoms in this molecule.
    :param attention_weights: A num_atoms x num_atoms PyTorch FloatTensor containing attention weights.
    """
    if type(smiles) == str:
        mol_name = smiles
        print('Saving {0} ({1} atoms)'.format(smiles, num_atoms))
        mol = Chem.MolFromSmiles(smiles)
    else:
        mol = smiles
        mol_name = Chem.MolToSmiles(mol)
        print('Saving Similarity map of molecule: {0} ({1} atoms)'.format(
            mol_name, num_atoms))

    smiles_viz_dir = viz_dir
    os.makedirs(smiles_viz_dir, exist_ok=True)
    atomSum_weights = np.zeros(num_atoms)

    # attention_weights = (attention_weights[4] + attention_weights[5]) / 2

    for a in range(num_atoms):
        a_weights = attention_weights[a].cpu().data.numpy()
        atomSum_weights += a_weights

    Amean_weight = atomSum_weights / num_atoms
    # print(Amean_weight)
    # 计算平均值和标准差
    mean_value = np.mean(Amean_weight)
    std_deviation = np.std(Amean_weight)

    # 根据平均值和标准差来确定阈值
    threshold0 = mean_value + 2 * std_deviation  # 例如，可以选择平均值加两倍标准差作为阈值
    threshold1 = mean_value - 2 * std_deviation 
    print(threshold0)
    print(threshold1)
    truncated_list = [i if i >= threshold1 else threshold1 for i in[num if num <= threshold0 else threshold0 for num in Amean_weight]]
    # print(truncated_list)
    Amean_weight = truncated_list
    nanMean = np.nanmean(Amean_weight)

    save_path = os.path.join(
        smiles_viz_dir, f'{mol_name.replace("/", "")}{prefix}.png')

    fig = SimilarityMaps.GetSimilarityMapFromWeights(mol, Amean_weight-nanMean,
                                                     alpha=0.3,
                                                     size=(300, 300))

    fig.savefig(save_path, bbox_inches='tight', dpi=300)
    plt.close(fig)

In [67]:
# from rdkit.Chem.Draw import SimilarityMaps
# from rdkit import Chem
# fig = SimilarityMaps.GetSimilarityMapFromWeights(Chem.MolFromSmiles("*C#*"), [1,2,1],
#                                                     alpha=0.3,
#                                                     size=(300, 300))

# fig.show()

In [2]:
import torch
attns = torch.load("/home/chenlidong/polyAttn/notebooks/attns_homo_0_3.pt",map_location=torch.device('cpu'))
smiles = "[*]Oc1ccc(S(=O)(=O)c2ccc(-c3ccc(-c4ccc(S(=O)(=O)c5ccc([*])cc5)cc4)cc3)cc2)cc1".replace(" ","")
atom_num = Chem.MolFromSmiles(smiles).GetNumAtoms()
idx = 6
atom_num

39

In [8]:
torch.sum(attns[6],dim=0)[:39,:39]

torch.Size([47, 47])

In [74]:
for i in range(8):
    visualize_atom_attention("/home/chenlidong/polyAttn/notebooks/",smiles,atom_num,attns[idx][i][:atom_num,:atom_num],str(i))

Saving [*]Oc1ccc(S(=O)(=O)c2ccc(-c3ccc(-c4ccc(S(=O)(=O)c5ccc([*])cc5)cc4)cc3)cc2)cc1 (39 atoms)
0.1236404109782598
-0.07235836336063144
Saving [*]Oc1ccc(S(=O)(=O)c2ccc(-c3ccc(-c4ccc(S(=O)(=O)c5ccc([*])cc5)cc4)cc3)cc2)cc1 (39 atoms)
0.12634801723545352
-0.07506596577598167
Saving [*]Oc1ccc(S(=O)(=O)c2ccc(-c3ccc(-c4ccc(S(=O)(=O)c5ccc([*])cc5)cc4)cc3)cc2)cc1 (39 atoms)
0.12516358889561982
-0.07388153688445062
Saving [*]Oc1ccc(S(=O)(=O)c2ccc(-c3ccc(-c4ccc(S(=O)(=O)c5ccc([*])cc5)cc4)cc3)cc2)cc1 (39 atoms)
0.14001820116475608
-0.08873614846269336
Saving [*]Oc1ccc(S(=O)(=O)c2ccc(-c3ccc(-c4ccc(S(=O)(=O)c5ccc([*])cc5)cc4)cc3)cc2)cc1 (39 atoms)
0.12230354212361148
-0.07102148825445416
Saving [*]Oc1ccc(S(=O)(=O)c2ccc(-c3ccc(-c4ccc(S(=O)(=O)c5ccc([*])cc5)cc4)cc3)cc2)cc1 (39 atoms)
0.14320447324140723
-0.09192242315895513
Saving [*]Oc1ccc(S(=O)(=O)c2ccc(-c3ccc(-c4ccc(S(=O)(=O)c5ccc([*])cc5)cc4)cc3)cc2)cc1 (39 atoms)
0.1226865486162081
-0.07140449755218202
Saving [*]Oc1ccc(S(=O)(=O)c2ccc(-c3ccc(-c4c

In [75]:
visualize_atom_attention("/home/chenlidong/polyAttn/notebooks/",smiles,atom_num,torch.sum(attns,dim=1)[idx][:atom_num,:atom_num].T,"sum")

Saving [*]Oc1ccc(S(=O)(=O)c2ccc(-c3ccc(-c4ccc(S(=O)(=O)c5ccc([*])cc5)cc4)cc3)cc2)cc1 (39 atoms)
0.20512821108671334
0.20512820177323646


In [ ]:
# [*]Oc1ccc(S(=O)(=O)c2ccc(-c3ccc(-c4ccc(S(=O)(=O)c5ccc([*])cc5)cc4)cc3)cc2)cc1